<a href="https://colab.research.google.com/github/Stefano-Previti/Ancient_to_Modern_Italian_Automatic_Translation/blob/main/Ancient_to_Modern_Italian_AT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0:IMPORTINGN LIBRARIES

In [78]:
!pip install -U gdown
!pip install -q transformers sentencepiece torch
!pip install -q  accelerate pandas
!pip install -q --upgrade bitsandbytes
!pip install fschat

In [80]:
import pandas as pd
import torch
import transformers
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from google.colab import drive
import bitsandbytes
from transformers import  BitsAndBytesConfig,AutoConfig,AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import StoppingCriteria
from huggingface_hub import login
import json
from fastchat.conversation import get_conv_template
import re
from sklearn.metrics import accuracy_score, cohen_kappa_score
from getpass import getpass


# 1: LOAD DATA

In [81]:
!gdown --fuzzy "https://drive.google.com/file/d/1SylTNYz4Ipv6GX75IcGEMTKa-nn1ke32/view?usp=sharing" -O dataset

Downloading...
From: https://drive.google.com/uc?id=1SylTNYz4Ipv6GX75IcGEMTKa-nn1ke32
To: /content/dataset
100% 14.6k/14.6k [00:00<00:00, 27.5MB/s]


In [ ]:
# Extract the data.
df = pd.read_csv('/content/dataset')
df.head()

,Author,Date,Region,Sentence
0,Brunetto Latini,1260-61,fior.,quella guerra ben fatta l' opera perché etc. E...
1,Bono Giamboni,1292,fior.,"crudele, e di tutte le colpe pigli vendetta, c..."
2,Valerio Massimo (red. V1,1336,fior.,Non d' altra forza d' animo fue ornato Ponzio ...
3,Lucano volg. (ed. Marinoni),1330/40,prat.,Se questo piace a tutti e se 'l tempo hae biso...
4,Brunetto Latini,1260-61,fior.,Officio di questa arte pare che sia dicere app...


In [ ]:
# Rename the sentence column to 'source'.
df = df.rename(columns={"Sentence": "source"})

# Select the 'source' column.
dataset = df[["source"]].dropna().reset_index(drop=True)

# Export to JSON Lines format.
dataset.to_json(
    "input.jsonl",
    orient="records",
    lines=True,
    force_ascii=False
)

print(f" Created input.jsonl with {len(dataset)} records.")


 Created input.jsonl with 97 records.


In [ ]:
drive.mount('/content/drive')
device = "cuda" if torch.cuda.is_available() else "cpu"

Mounted at /content/drive


In [ ]:
!head -n 100 input.jsonl


{"source":"quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno"}
{"source":"crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi."}
{"source":"Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere."}
{"source":"Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati."}
{"source":"Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire."}
{"source":"Ecco e larghi ventipiovoli caggiono delle risolute nebbie; e potresti credere che tutto il cielo cadesse nel mare"}
{"source":"Però che or chi spererebbe quello che eziandio questi che non vogliono ancora credere in Cristo, già veggiono con noi, e perché nol possono negare, stridono colli denti."}
{"source":"I vendimenti de' morti et le presure

# 2: TRANSFORMER BASED MACHINE TRANSLATION

## 2.0: TRANSLATION UTILITY FUNCTION

In [ ]:
def translation(text, src_lang, tgt_lang,num_beams,max_length,no_repeat_ngram_size,length_penalty,repetition_penalty,early_stopping,use_cache):

   # Configuration of the tokenizer
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    forced_bos = tokenizer.lang_code_to_id[tgt_lang]

    # Tokenizer
    inputs = tokenizer(
        text=text,
        return_tensors="pt",
        truncation=True,
        max_length=max_length
    ).to(device)

    # Generation
    with torch.no_grad():
      outputs = model.generate(
          **inputs,
          forced_bos_token_id=forced_bos,
          num_beams=num_beams, # use beam search for broader hypothesis exploration
          max_length=max_length,
          no_repeat_ngram_size=no_repeat_ngram_size, # prevent the model from repeating any n-gram in the output
          length_penalty=length_penalty, # encourage longer and more complete outputs (penalize too-short sequences)
          repetition_penalty=repetition_penalty, # heavily penalize repetition of the same tokens
          early_stopping=early_stopping,
          use_cache=use_cache
      )
    # Decode to string
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

## 2.1:mBART-50

mBART-50 is a multilingual Sequence-to-Sequence model pre-trained using the "Multilingual Denoising Pretraining" objective.

We tried to use a larger hypothesis space with 8 beams , a lenght-penalty, a repetition penalty and we disallowed any repeatition of 3-gram in the output .

Despite this, the model has some problem in the final output translating improperly some words in english instead of modern italian.

Reference: https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt


In [ ]:
# Load the pre-trained mBART-50 model and tokenizer
MODEL = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(MODEL)
model = MBartForConditionalGeneration.from_pretrained(MODEL).to(device)
model.eval()


# Apply the translation to each source sentence
df = pd.read_json("input.jsonl", lines=True)
df["translation"] = df["source"].apply(lambda txt: translation(text=txt,
                                                               src_lang="it_IT",
                                                               tgt_lang="it_IT",
                                                               num_beams=8,
                                                               max_length=256,
                                                               no_repeat_ngram_size=3,
                                                               length_penalty=1.2,
                                                               repetition_penalty=2.0,
                                                               early_stopping=True,
                                                               use_cache=True))

# Export to a new JSONL file.
output_path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50.jsonl'
df.to_json(
    output_path,
    orient='records',
    lines=True,
    force_ascii=False
)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")


SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: La guerra ha fatto bene, perché etc. E dall'altra parte Aiaces era un bravo guerriero, e brava per le armi, ma non pieno di grande senno.
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: crudele, e per tutte le colpe vendetta, come dice la legge, e da cavaliere perdon che pecchi.
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Non c'è altra forza d' animosità che è stato venerato il Ponzio dell'Umiliare, un romano cavaliere.
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
TRANSLATION: Se questo piace a tutti e se il tempo ha bisogno di Pompei per cavaliere e non per compagno, n

## 2.2: M2M100

M2M100 is a multilingual encoder-decoder (seq-to-seq) model trained for Many-to-Many multilingual translation.

We set the same parameters as before for the generation phase.

Here we didn't suffer about the swithcing in other language in the output.

Reference: https://huggingface.co/docs/transformers/model_doc/m2m_100

In [ ]:
# Model and tokenizer,
MODEL = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(MODEL)
model = M2M100ForConditionalGeneration.from_pretrained(MODEL).to(device)
model.eval()

# Read file.
df = pd.read_json("input.jsonl", lines=True)

# Translate and saving.
df["translation"] = df["source"].apply(lambda txt: translation(text=txt,
                                                               src_lang="it",
                                                               tgt_lang="it",
                                                               num_beams=8,
                                                               max_length=256,
                                                               no_repeat_ngram_size=3,
                                                               length_penalty=1.2,
                                                               repetition_penalty=2.0,
                                                               early_stopping=True,
                                                               use_cache=True))

output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100.jsonl"
df.to_json(
    output_path,
    orient="records",
    lines=True,
    force_ascii=False
)



In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: E dall'altra parte Aiaces era un cavaliere franco e prodo alle armi, di grande fortuna, ma non era pieno di grande seno.
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: crudele, e di tutti i peccati piglia vendetta, come dice la legge, e a nove cavalieri perdono che peccano.
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Nessuna altra forza d'anima fu ornato Ponzio Aufidiano, cavaliere romano.
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
TRANSLATION: Se questo piace a tutti e se il tempo ha bisogno di avere Pompeo per cavaliere e non per compagno, non riterò più i fatti.
S

# 3: LLM

## 3.0: UTILITY FUNCTIONS

Here are showed the **quantization** in order to run 7B model on colab in relatively short time.

Then it's implemented the generation function of the LLM-based translators, with the utility "NewLineStoppingCriteria"

Finally, the list of system prompts and input/output prompts.

The few shot examples will be of two types:
- **Vanilla**: the examples are simple pairs input/output and
- **Reasoning**: the examples are augmented with step-by-step reasoning, in order to obtain better performance in terms of faithfulness with respect to the original meaning.

In [ ]:
# Login to hugging face.
api_token = getpass("Insert token: ")
login(api_token)

# Configuration for the quantization in order to run in Colab.
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [83]:
# Help function to avoid new line in the final translation.
class NewlineStoppingCriteria(StoppingCriteria):
    def __init__(self, tokenizer, initial_input_length, num_newlines):
        super().__init__()
        self.tokenizer = tokenizer
        self.initial_input_length = initial_input_length
        self.num_newlines = num_newlines

    def __call__(self, input_ids, scores, **kwargs):
        generated_ids = input_ids[0, self.initial_input_length:]
        text = self.tokenizer.decode(generated_ids, skip_special_tokens=True)

        # Counting number of new lines to stop.
        return text.count("\n") >= self.num_newlines

# Help function to build the prompt and translate.
def icl_translate(text,system_prompt, examples,max_new_tokens,do_sample,temperature,top_p,repetition_penalty,num,mistral_bool,reasoning_bool):

    # Add examples for in context learning (empty list for zero shot).
    conv = [system_prompt]+ examples + [
        {"role": "user", "content": f"Input: {text}\nOutput:"}
    ]

    # Chat template.
    inputs = tokenizer.apply_chat_template(
        conv,
        return_tensors="pt",
        return_dict=True
    ).to(model.device)

    input_len = inputs["input_ids"].shape[-1]

    # Generation.
    with torch.no_grad():
      outputs = model.generate(
          **inputs,
          pad_token_id=tokenizer.pad_token_id,
          max_new_tokens=max_new_tokens,
          do_sample=do_sample,
          temperature=temperature, # moderate diversity in the translation.
          top_p=top_p,  # picking the top-p% best
          repetition_penalty=repetition_penalty, # penalty to avoid useless repetition in the translation.
          stopping_criteria=[NewlineStoppingCriteria(tokenizer,  input_len, num)] # Stopping when a number_of_new_line=num is encountered in the output.
      )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if reasoning_bool:
        for sep in reversed(["2. Translation:", "2. Traduzione:"]):
            if sep in decoded:
                return decoded.rsplit(sep, 1)[-1].strip()
        return decoded.strip()

    # Extract last Output with a boolean for the extraction of the translation of the two LLMs.
    if mistral_bool:
      return decoded.split("Output:")[-1].strip()
    else:
      return decoded.rsplit("<|assistant|>", 1)[-1].strip()


In [84]:
# System message for zero shot and vanilla in context learning.
system_prompt = {
    "role": "system",
    "content": (
        "You are an expert translator who converts archaic Italian (1250–1400) into fluent, modern Italian.\n"
        "Your goal is to preserve the **original meaning** of the source text as faithfully as possible.\n"
        "**Your only objective is to produce exclusively the Italian translation: nothing else except the translation must be generated!**\n\n"
        "**IMPORTANT :**\n"
        "1. The translation must be written only in modern Italian.\n"
        "2. Any other language is prohibited.\n"
        "3. Avoid any kind of repetition in the translation.\n"
        "4. Do **NOT** repeat the instructions.\n"
        "5. Comments, notes, or explanations in brackets are strictly prohibited (e.g., “(this word means...)”, “(Note:...)”, “(Translated from...)”, “(...)”).\n"
        "6. Do **NOT** add anything else — no comments, no notes, no introductions and no explanations (e.g., “Translated in...”, “In modern Italian...”).\n"
        "7. Reorganize the sentence structure if necessary to ensure fluency and clarity in modern Italian. Preserve meaning, but adapt archaic syntax into a natural, modern phrasing.\n"
        "8. Show **DIRECTLY** the modern italian translation of the sentence written by the user. Do NOT add anything before or after it.\n\n"
    )
}


# System message for the zephyr model: similar as before but adding an explicit instruction to avoid the spanish translation.
zephyr_system_prompt = {
    "role": "system",
    "content": (
        "You are an expert translator who converts archaic Italian (1250–1400) into fluent, modern Italian.\n"
        "Your goal is to preserve the **original meaning** of the source text as faithfully as possible.\n"
        "**Your only objective is to produce exclusively the Italian translation: nothing else except the translation must be generated!**\n\n"

        "**STRICT LANGUAGE POLICY:**\n"
        "1. The translation must be written **exclusively in modern Italian**.\n"
        "2. **Spanish is strictly prohibited in all forms** — any use of Spanish words, phrases or expressions (such as *de, que, en, pero, muy, con, para, entonces, este, aquel*) is a **critical error**.\n"
        "3. **Even isolated Spanish words** will be considered a **big mistake**.\n"
        "4. Language switching is **not allowed**. Do not mix or alternate between Italian and Spanish.\n"
        "5. The only language permitted is **Italian**. Any other language is forbidden.\n\n"

        "**TRANSLATION RULES:**\n"
        "6. Avoid any kind of repetition in the translation.\n"
        "7. Do **NOT** repeat the instructions.\n"
        "8. Comments, notes, or explanations in brackets are strictly prohibited (e.g., “(this word means...)”, “(Note:...)”, “(Translated from...)”, “(Modern italian..)”).\n"
        "9. Do **NOT** add anything else — no comments, no notes, no introductions and no explanations (e.g., “Translated in...”, “In modern Italian...”).\n"
        "10. Reorganize the sentence structure if necessary to ensure fluency and clarity in modern Italian. Preserve meaning, but adapt archaic syntax into a natural, modern phrasing.\n"
        "11. Show **DIRECTLY** the modern italian translation of the sentence written by the user. Do NOT add anything before or after it.\n\n"
    )
}


# System message for in context learning with reasoning.
reasoning_system_prompt = {
    "role": "system",
    "content": (
        "You are an expert translator who converts archaic Italian (1250–1400) into fluent, elegant and natural modern Italian.\n"
        "Your goal is to preserve the **original meaning** of the source text as faithfully as possible, while making the output sound like it was written today.\n\n"

        "Before the translation, you must analyze the sentence step by step, considering:\n"
        "1. The global meaning of the sentence.\n"
        "2. Any poetic meaning that must be preserved.\n"
        "3. The overall **syntactic** and **grammatical** structure.\n\n"

         "**Important instructions:**\n"
        " 1. Reasoning phase instructions:\n"
        "- **IMPORTANT**: The reasoning must consist of **maximum 4 steps**!\n"
        "- Each step must correspond to a specific **significant** portion of the sentence.\n"
        "- Each step must focus only on the most **relevant and informative features** of the portion analyzed. Avoid redundant or trivial details.\n"
        "- **Each step must be written as a compact,concise paragraph of 1–2 full sentences** .In each sentence do not join multiple clauses with semicolons or excessive commas.\n"
        "- **IMPORTANT**: Each individual sentence within a step must be **short**, direct, and self-contained.\n"
        "- **Strictly prohibited WITHIN any step**: bullet points (•, -, *, etc.), dashes (–), numbered sub-lists (1., 2.), lettered sub-lists (a), b)), or any other kind of list.\n"
        "- A word-by-word analysis is **forbidden**.\n\n"

        "2. Translation phase instructions:"
        "- The translation must be written **exclusively in modern Italian**.\n"
        "- The only language permitted is Italian. Any other language is **strictly prohibited**. Any mixing or alternation between italian and any other language is a **critical error**.\n"
        "- Avoid any kind of repetition in the translation.\n"
        "- Do **NOT** repeat the instructions.\n"
        "- In the translation comments, notes, or explanations in brackets are prohibited (e.g., “(this word means...)”, “(this sentence means...)”, “(NB:...)”, “(Note:...)”, “(Translated from...)”).\n"
        "- Do **NOT** add anything else — no comments, no notes, no introductions and no explanations (e.g., “Translated in...”, “In modern Italian...”).\n"
        "- **The only objective,in this phase,is to produce exclusively the Italian translation: nothing else except the translation must be generated!**\n"
        "- Reorganize the sentence structure if necessary to ensure fluency and clarity in modern Italian. Preserve meaning, but adapt archaic syntax into a natural, modern phrasing.\n\n"


        "The output must follow this format:\n"
        "1. Reasoning: a concise,synthetic, step-by-step explanation.\n"
        "2. Translation: the final modern Italian version of the sentence.\n"
    )
}

# In-context learning examples with "role" and "content".
examples = [
    {"role": "user", "content": "Input: Quivi sospiri, pianti e alti guai risonavan per l’aere sanza stelle.\nOutput:"},
    {"role": "assistant", "content": "In quel luogo risuonavano sospiri, pianti e grida di dolore nell’aria oscura, priva di stelle.\n"},
    {"role": "user", "content": "Input: O voi ch’avete li ’ntelletti sani, mirate la dottrina che s’asconde sotto ’l velame de li versi strani.\nOutput:"},
    {"role": "assistant", "content": "Voi che possedete intelletto sano, osservate la dottrina nascosta dietro il velo di questi versi insoliti.\n"},
    {"role": "user", "content": "Input: Solo et pensoso i più deserti campi vo mesurando a passi tardi e lenti.\nOutput:"},
    {"role": "assistant", "content": "Solo e pensieroso, attraverso i campi più desolati avanzo con passi lenti e ponderati.\n"},
    {"role": "user", "content": "Input: Voi ch’ascoltate in rime sparse il suono, di quei sospiri ond’io nudriva ’l core.\nOutput:"},
    {"role": "assistant", "content": "Voi che udite il suono di queste rime sparse, e quei sospiri che un tempo nutrivano il mio cuore.\n"},
    {"role": "user", "content": "Input: In quella prima sera, la bella Fiammetta, vedendo uno dei più gentili e onesti giovani di compagnia, prese gran piacere di lui.\nOutput:"},
    {"role": "assistant", "content": "Quella prima sera, la bella Fiammetta, vedendo uno dei giovani più gentili e onesti del gruppo, provò grande simpatia per lui.\n"},
]


# In contexr learning with reasoning about the meaning of the sentences.
reasoning_examples = [

    {"role": "user", "content": "Input: Quivi sospiri, pianti e alti guai risonavan per l’aere sanza stelle.\nOutput:"},
    {"role": "assistant", "content":
        "1. Reasoning:"
        "Step 1. Il termine 'Quivi' introduce un luogo preciso e lontano, evocando fin da subito un’atmosfera oscura e desolata.\n"
        "Step 2. Le espressioni 'sospiri, pianti e alti guai' trasmettono una condizione di dolore intenso e collettivo, accentuata dall’uso della parola 'alti' per indicare la gravità delle sofferenze.\n"
        "Step 3. Il verbo 'risonavan' sottolinea la continuità e la forza di questi lamenti, che sembrano propagarsi senza tregua.\n"
        "Step 4. L’immagine dell’aria 'sanza stelle' rafforza la sensazione di abbandono e oscurità, creando un’atmosfera priva di speranza.\n"
        "2. Translation: In quel luogo risuonavano sospiri, pianti e grida di dolore nell’aria oscura, priva di stelle.\n"
    },

    {"role": "user", "content": "Input: O voi ch’avete li ’ntelletti sani, mirate la dottrina che s’asconde sotto ’l velame de li versi strani.\nOutput:"},
    {"role": "assistant", "content":
        "1. Reasoning:"
        "Step 1. L’incipit 'O voi ch’avete li ’ntelletti sani' si rivolge a un pubblico colto e attento, invitandolo a un’esplorazione profonda del testo.\n"
        "Step 2. Il verbo 'mirate' richiama un’attenzione attiva e riflessiva, suggerendo che l’interpretazione richiede uno sforzo consapevole.\n"
        "Step 3. La 'dottrina che s’asconde sotto ’l velame' allude a un significato nascosto che si cela dietro le immagini poetiche.\n"
        "Step 4. L’espressione 'versi strani' indica una forma espressiva complessa, scelta per velare concetti profondi.\n"
        "2. Translation: Voi che possedete intelletto sano, osservate la dottrina nascosta dietro il velo di questi versi insoliti.\n"
    },

    {"role": "user", "content": "Input: Solo et pensoso i più deserti campi vo mesurando a passi tardi e lenti.\nOutput:"},
    {"role": "assistant", "content":
        "1. Reasoning:"
        "Step 1. L’espressione 'Solo et pensoso' descrive una condizione di solitudine interiore e meditazione malinconica.\n"
        "Step 2. 'I più deserti campi' rappresentano luoghi abbandonati e remoti, scelti proprio per evitare ogni contatto umano.\n"
        "Step 3. Il verbo 'vo mesurando' suggerisce un camminare lento e riflessivo, come se il protagonista stesse misurando lo spazio per trovare equilibrio.\n"
        "Step 4. L’espressione 'passi tardi e lenti' rafforza il senso di stanchezza e di introspezione profonda.\n"
        "2. Translation: Solo e pensieroso, attraverso i campi più desolati avanzo con passi lenti e ponderati.\n"
    },

    {"role": "user", "content": "Input: Voi ch’ascoltate in rime sparse il suono, di quei sospiri ond’io nudriva ’l core.\nOutput:"},
    {"role": "assistant", "content":
        "1. Reasoning:"
        "Step 1. L’invocazione iniziale “Voi ch’ascoltate” stabilisce un legame diretto con il lettore e introduce un tono confidenziale e personale.\n"
        "Step 2. L’espressione “in rime sparse” fa riferimento a componimenti distribuiti nel tempo e nel contenuto, che testimoniano un vissuto frammentato.\n"
        "Step 3. “Il suono di quei sospiri” suggerisce che il poeta ha trasformato emozioni profonde in versi e che quei sospiri erano il nutrimento del suo cuore.\n"
        "Step 4. “Ond’io nudriva ’l core” rafforza l’idea che l’amore fosse un’esperienza intensa e sofferta, interiorizzata profondamente come alimento dell’anima.\n"
        "2. Translation: Voi che udite il suono di queste rime sparse, e quei sospiri che un tempo nutrivano il mio cuore.\n"
    },

    {"role": "user", "content": "Input: In quella prima sera, la bella Fiammetta, vedendo uno dei più gentili e onesti giovani di compagnia, prese gran piacere di lui.\nOutput:"},
    {"role": "assistant", "content":
        "1. Reasoning:"
        "Step 1. L’espressione 'In quella prima sera' introduce l’inizio di un evento significativo in un’atmosfera precisa e iniziale.\n"
        "Step 2. Il personaggio di 'la bella Fiammetta' viene presentato con grazia e centralità narrativa, sottolineandone l’impatto.\n"
        "Step 3. La vista di 'uno dei più gentili e onesti giovani' suggerisce una prima attrazione fondata su qualità morali oltre che estetici.\n"
        "Step 4. L'espressione 'prese gran piacere di lui' indica la nascita di un sentimento affettuoso, espresso con delicatezza e misura.\n"
        "2. Translation: Quella prima sera, la bella Fiammetta, vedendo uno dei giovani più gentili e onesti del gruppo, provò grande simpatia per lui.\n"
    },
]



## 3.1: MISTRAL-7B INSTRUCT V-03

The Mistral-7B-Instruct-v0.3 Large Language Model (LLM) is an instruct fine-tuned version of the Mistral-7B-v0.3.

In the following sections we will use it in a zero shot mode,but also in a few shot mode,due to the capacity of the model to follow instructions. The few shot will be of in the two types: vanilla and reasoning.

Reference:https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [ ]:
# Model and tokenizer.
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config,
)
model.config.pad_token_id = tokenizer.pad_token_id
model.eval()



### 3.1.1: MISTRAL-7B INSTRUCT V-03 ZERO SHOT

In this first section the model will be in "zero shot" mode,with just the system prompt explaining the task to be performed.


In [ ]:
# Load input.
df = pd.read_json("input.jsonl", lines=True)

# Translate and save.
df_ = df.copy()
df_["translation"] = df_["source"].apply(lambda s: icl_translate(s,system_prompt, examples=[],max_new_tokens=150,do_sample=True,temperature=0.75,top_p=0.95,repetition_penalty=1.2,num=1,mistral_bool=True,reasoning_bool=False))
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot.jsonl"
df_.to_json(output_path, orient="records", lines=True, force_ascii=False)

In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella guerra fu bene condotta per ragioni complesse et Aiace era un cavaliere francese coraggioso e abile nell'utilizzo delle armi, ma mancava di profonda saggezza
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: crudele è e prende vendetta per ogni reato, come la legge prevede, e non perdonerà ad alcun cavalier quando commette un errore.
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Non fu adornato con altro spirito Ponzio Afidiano, un cavaliere romano.
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
TRANSLATION: Se tutti lo trovano piacevole e il tempo necessita di Pompeo 

### 3.1.2: MISTRAL-7B INSTRUCT V-03 VANILLA FIVE SHOT

Now it will be in a "five shot" mode picking all the "vanilla" predefined examples.

In [ ]:
# Translate and save.
df_ = df.copy()
df_["translation"] = df_["source"].apply(lambda s: icl_translate(s,system_prompt, examples[:10],max_new_tokens=150,do_sample=True,temperature=0.75,top_p=0.95,repetition_penalty=1.2,num=1,mistral_bool=True,reasoning_bool=False))
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot.jsonl"
df_.to_json(output_path, orient="records", lines=True, force_ascii=False)

In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella guerra fu condotta con grande abilità, poiché [l'azione] aveva una buona ragione. Da un lato c'era Achille, un cavalieri francese valente e coraggioso nelle armi, molto capace, ma non troppo astuto.
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: Crudelmente, prendeva rivalsa di ogni colpa commessa, secondo quanto stabilito dalla legge, e non perdonava ad alcun cavalieri che peccassevano.
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Non fu adornato da alcun'altra forza d'animo Ponzio Aufidiano, cavaliere romano.
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
TRANSLA

### 3.1.3: MISTRAL-7B INSTRUCT V-03 FIVE SHOT-> REASONING AUGMENATION

In this final section the model will be in a five shot mode with the reasoning augmentation, changing the system prompt and the examples to the "reasoning" ones.

In [ ]:
# Translate and save.
df_ = df.copy()
df_["translation"] = df_["source"].apply(lambda s: icl_translate(s,reasoning_system_prompt, reasoning_examples[:10],max_new_tokens=450,do_sample=True,temperature=0.7,top_p=0.9,repetition_penalty=1.2,num=15,mistral_bool=True,reasoning_bool=True))
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot.jsonl"
df_.to_json(output_path, orient="records", lines=True, force_ascii=False)

In [ ]:

'''
Cleaning:
Even if there is already the new line stopping criteria,it's impossible(in this case)
to predict the precise number of new line for every output,so we are performing a further cleaning.
'''
output_path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot.jsonl'
df_new = pd.read_json(output_path, lines=True)
df_new["translation"] = df_new["translation"].apply(lambda txt: txt.split('\n', 1)[0].strip())
df_new.to_json(output_path, orient="records", lines=True, force_ascii=False)



In [ ]:
# Print the translations.
output_path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot.jsonl'
df_final = pd.read_json(output_path, lines=True)

for idx, row in df_final.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella guerra fu combattuta per ragioni importanti ed efficaci. Tuttavia, dalla parte opposta vi era Achille, un cavalieri francese coraggioso e molto abilmente armato, ma non sempre intelligente.
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: La sua natura era brutalmente severa, puniva tutti gli errori secondo quanto imposti dalla legge, e non aveva mai compiacenza verso coloro che peccavano contro di lui.
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Non venne decorato con altre motivazioni Ponzio Aufidiano, cavalieri romani.
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fat

## 3.2: ZEPHYR-7B-BETA

Zephyr is a series of language models that are trained to act as helpful assistants.

The model was initially fine-tuned on a filtered and preprocessed of the UltraChat dataset, which contains a diverse range of synthetic dialogues generated by ChatGPT. Then it was further aligned with TRL's DPOTrainer on the openbmb/UltraFeedback dataset, which contains 64k prompts and model completions that are ranked by GPT-4.

Given its high capacity in chat-like conversation, we decide to use it with an in context learning strategy where,as before, the input/output examples are showed as a dialogue between the user and the model.

The following three section will be: "vanilla three shot" mode, "vanilla five shot" and "reasoning five shot".

Note: the "zero shot" mode is eliminated since there were to much spanish switching in the final output.

Reference: https://huggingface.co/HuggingFaceH4/zephyr-7b-beta

In [ ]:
# Model and tokenizer.
model_id = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config,
)
model.config.pad_token_id = tokenizer.pad_token_id
model.eval()



### 3.2.1: ZEPHYR-7B-BETA VANILLA THREE SHOT

Here,in the first section, the model is in a "three shot" mode picking the system prompt with the first 3 vanilla input/output examples.

In [ ]:
# Load input.
df = pd.read_json("input.jsonl", lines=True)

# Translate and save.
df_ = df.copy()
df_["translation"] = df_["source"].apply(lambda s: icl_translate(s,zephyr_system_prompt, examples[:6],max_new_tokens=150,do_sample=True,temperature=0.75,top_p=0.95,repetition_penalty=1.2,num=2,mistral_bool=False,reasoning_bool=False))
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot.jsonl"
df_.to_json(output_path, orient="records", lines=True, force_ascii=False)

In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella Guerra fu un'operazione molto bene realizzata e la sua efficacia deriva da vari motivi, tra cui ... E d'altro canto Èaco, un cavaliere francese, era un uomo valoroso e pronto alla battaglia, di grandissimo valore militare, ma non raggiungeva la massima saggezza intellettuale.
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: spietata, e di ogni colpa prende la vendetta, come prevede la legge, e perdona a ciascun cavalieri peccatore.
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Non fu decorato con altre onorificenze da parte della Repubblica Romana Alfio Ponzio, valente cavaliero.
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompei

### 3.2.2: ZEPHYR-7B-BETA VANILLA FIVE SHOT

Here the model is in a "five shot" mode picking all the "vanila" input/output examples.

In [ ]:
# Load input.
df = pd.read_json("input.jsonl", lines=True)

# Translate and save.
df_ = df.copy()
df_["translation"] = df_["source"].apply(lambda s: icl_translate(s,zephyr_system_prompt, examples[:10],max_new_tokens=150,do_sample=True,temperature=0.75,top_p=0.95,repetition_penalty=1.2,num=2,mistral_bool=False,reasoning_bool=False))
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot.jsonl"
df_.to_json(output_path, orient="records", lines=True, force_ascii=False)

In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quell'ottava guerra fu una opera eccellente per cui si può dire "etc.", poiché (da parte opposta), Ettore era un valente cavalieri francese e coraggioso sul campo di battaglia, di gran lunga; tuttavia, non disponeva di grandissima saggezza.
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: savero severamente, e di ogni colpa si prende la vendetta, come prescrive la legge, senza perdonomare alcun cavalieri peccatore.
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Ponzio Aufidiano, romano cavalieri celebre, non fu decorato per altre cause se non per il suo spirito eroico.
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e n

### 3.2.3: ZEPHYR-7B-BETA FIVE SHOT-> REASONING AUGMENTATION

In this final section the model will be in a five shot mode with the reasoning augmentation, changing the system prompt and the examples to the "reasoning" ones.

In [ ]:
# Load input.
df = pd.read_json("input.jsonl", lines=True)

# Translate and save.
df_ = df.copy()
df_["translation"] = df_["source"].apply(lambda s: icl_translate(s,reasoning_system_prompt, reasoning_examples[:10],max_new_tokens=450,do_sample=True,temperature=0.7,top_p=0.9,repetition_penalty=1.2,num=15,mistral_bool=False,reasoning_bool=True))
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot.jsonl"
df_.to_json(output_path, orient="records", lines=True, force_ascii=False)

In [ ]:

'''
Cleaning:
Even if there is already the new line stopping criteria,it's impossible(in this case)
to predict the precise number of new line for every output,so we are performing a further cleaning.
'''
output_path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot.jsonl'
df_new = pd.read_json(output_path, lines=True)
df_new["translation"] = df_new["translation"].apply(lambda txt: txt.split('\n', 1)[0].strip())
df_new.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot.jsonl'
df_final = pd.read_json(path, lines=True)

for idx, row in df_final.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quell'operazione bellica fu efficacemente condotta; ecco dalla parte opposta vi era Ayace, cavaliero francese valorosissimo nelle armi, molto deciso nel servizio, ma non dotato di grande saggezza mentale.
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: Crudelmente, prende la sua vendetta contro tutti gli errori, come prescrive la legge, e nessun cavaliere può ottenere perdono per qualsiasi offesa.
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Non fu decorato con alcun'altra gloria Ponzio Aufidiano, celebre cavaliere romano.
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
TRA

# 4: LLM AS A JUDGE-> PROMETHEUS

Prometheus 2 is a language model using Mistral-Instruct as a base model. It is fine-tuned on 100K feedback within the Feedback Collection and 200K feedback within the Preference Collection. It is a specifically model fine tuned for the judging task.

Reference: https://huggingface.co/prometheus-eval/prometheus-7b-v2.0

### 4.1: GOLD REFERENCE

The gold reference are human manually written. Even with the possibility to avoid the reference, we decide to add them (trying to be as accurate and complete as possible), in order to get more robust and complete result.

In [ ]:
# Manually written gold reference.
gold_reference = [
    {"source": "quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno",
     "reference": "Quella guerra fu ben condotta, l’opera riuscì, ecc. E dall’altra parte Aiace era un cavaliere franco e prode all’arme, di gran guisa, ma non era pieno di grande senno."},
    {"source": "crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.",
     "reference": "Crudele, e di tutte le colpe si vendicava, come prescrive la legge, e a nessun cavaliere perdona chi pecca."},
    {"source": "Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.",
     "reference": "Non d’altra forza d’animo fu ornato Ponzio Aufidiano, romano cavaliere."},
    {"source": "Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.",
     "reference": "Se questo piace a tutti e se il tempo ha bisogno di avere Pompeio per cavaliere e non per compagno, non tratterrò oltre il destino."},
    {"source": "Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire.",
     "reference": "Il compito di quest’arte sembra che sia parlare appositamente per far credere; il fine è far credere con le parole."},
    {"source": "Ecco e larghi ventipiovoli caggiono delle risolute nebbie; e potresti credere che tutto il cielo cadesse nel mare",
     "reference": "Ecco, ampie piogge ventose cadono dalle nebbie dissolte; potresti credere che tutto il cielo cadesse nel mare."},
    {"source": "Però che or chi spererebbe quello che eziandio questi che non vogliono ancora credere in Cristo, già veggiono con noi, e perché nol possono negare, stridono colli denti.",
     "reference": "Poiché ora chi spererebbe ciò che perfino quelli che ancora non vogliono credere in Cristo già vedono con noi, e poiché non lo possono negare, digrignano i denti."},
    {"source": "I vendimenti de' morti et le presure de' vivi fece la frode d'uno feroce re.",
     "reference": "Le spoliazioni dei morti e le catture dei vivi furono frutto della frode di un re feroce."},
    {"source": "Acciocché quegli, il quale ora per le sue gran reità è feroce e onorevole, egli d'ogni male afflitto e tormentato della impietà verso il mio padre.",
     "reference": "Affinché colui, che ora per le sue grandi colpe è feroce e onorato, sia afflitto da ogni male e tormentato per l’empietà verso mio padre."},
    {"source": "Gli uomini spessamente a stare fermi nella bugia incontra la verità.",
     "reference": "Gli uomini, stando fermi spesso nella bugia, finiscono per incontrare la verità."},
    {"source": "Marco Cornelio ch'era de' dieci compagni, studiosamente  si riservò di parlare all'ultimo.",
     "reference": "Marco Cornelio, che era uno dei dieci compagni, si riservò con cura di parlare per ultimo."},
    {"source": "cose ch'io sapeva che erano fatte in Italia.",
     "reference": "Cose che sapevo essere avvenute in Italia."},
    {"source": "Corbio nipote d' Ortensio menò sua vita più bassa e più viziosa",
     "reference": "Corbio, nipote di Ortensio, condusse una vita più bassa e più viziosa."},
    {"source": "Altressì uno amante chiamando merzé alla sua donna dice parole e ragioni molte, et ella si difende in suo dire.",
     "reference": "Così pure un amante, chiedendo pietà alla sua donna, pronuncia molte parole e argomenti, ed ella si difende con le sue parole."},
    {"source": "Io mi ricordo (ch. 347) che essendo adirato scapigliai la mia donna. Ohi, quanti dì questa ira mi tolse!",
     "reference": "Ricordo (ch. 347) che, preso dall’ira, scapigliai la mia donna. Ah, quanti giorni mi costò quell’ira!"},
    {"source": "colui del quale tu tti solevi dolere ch' era amante della tua donna;",
     "reference": "Colui di cui ti lamentavi perché era amante della tua donna;"},
    {"source": "Ma no sapeano già le nomora di coloro dela congiurazione, ché la donna no nominava già li nomi.",
     "reference": "Ma non conoscevano ancora i nomi dei membri della congiura, perché la donna non li pronunciava."},
    {"source": "Creti?  Certo quand'elli si mosse, elli ti dixe: \"O fedele mia donna, fa' che in mio luogo ti sia racomandato il nostro hoste troiano\".",
     "reference": "Lo credi? Certo, quando egli si mosse, ti disse: «O mia fedele donna, fa’ che, al posto mio, sia raccomandato il nostro ospite troiano»."},
    {"source": "A Milano fue ripressa la malvagità d' una donna in simile bugìa, nel tempo medesimo di questo signore della republica, in questo modo: ",
     "reference": "A Milano la malvagità di una donna fu smascherata per una menzogna simile, nello stesso periodo di questo signore della repubblica, in questo modo:"},
    {"source": "E lamentavansi dell'iniquità d'Appio, e ripiagnevano la malavventurata beltà della pulcella e la necessità del padre.",
     "reference": "E si lamentavano dell’iniquità di Appio e piangevano di nuovo la sventurata bellezza della fanciulla e la costrizione del padre."}
]

adaptive_gold_reference = [
    {"source": "quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno",
     "reference": "Quella guerra fu portata a termine con successo; l’impresa riuscì. Aiace, invece, era un cavaliere leale e coraggioso in battaglia, imponente nella figura, ma non particolarmente saggio."},
    {"source": "crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.",
     "reference": "Era crudele: si vendicava di ogni colpa, come prescritto dalla legge, e non perdonava nessun cavaliere colpevole."},
    {"source": "Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.",
     "reference": "Ponzio Aufidiano, cavaliere romano, non possedeva alcun vigore d’animo straordinario."},
    {"source": "Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.",
     "reference": "Se tutti approvano e se il momento richiede che Pompeo sia un cavaliere e non un compagno, allora non ostacolerò più il destino."},
    {"source": "Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire.",
     "reference": "Il compito di quest’arte sembra essere quello di parlare in modo efficace per convincere; il suo scopo è far credere attraverso le parole."},
    {"source": "Ecco e larghi ventipiovoli caggiono delle risolute nebbie; e potresti credere che tutto il cielo cadesse nel mare",
     "reference": "Ecco, forti piogge spinte dal vento scendono dalle nebbie ormai dissolte; potresti pensare che tutto il cielo stia cadendo nel mare."},
    {"source": "Però che or chi spererebbe quello che eziandio questi che non vogliono ancora credere in Cristo, già veggiono con noi, e perché nol possono negare, stridono colli denti.",
     "reference": "Poiché oggi chi potrebbe ancora sperare in ciò che perfino coloro che non vogliono credere in Cristo già vedono insieme a noi e, non potendolo negare, digrignano i denti."},
    {"source": "I vendimenti de' morti et le presure de' vivi fece la frode d'uno feroce re.",
     "reference": "Le spogliazioni dei caduti e la cattura dei vivi furono causate dall’inganno di un re crudele."},
    {"source": "Acciocché quegli, il quale ora per le sue gran reità è feroce e onorevole, egli d'ogni male afflitto e tormentato della impietà verso il mio padre.",
     "reference": "Così colui che ora, nonostante le sue colpe, è temuto e onorabile, venga colpito da ogni sciagura e tormentato per aver mancato di rispetto a mio padre."},
    {"source": "Gli uomini spessamente a stare fermi nella bugia incontra la verità.",
     "reference": "Spesso, chi si ostina a restare nella menzogna finisce per scontrarsi con la verità."},
    {"source": "Marco Cornelio ch'era de' dieci compagni, studiosamente  si riservò di parlare all'ultimo.",
     "reference": "Marco Cornelio, uno dei dieci compagni, scelse con cura di essere l’ultimo a parlare."},
    {"source": "cose ch'io sapeva che erano fatte in Italia.",
     "reference": "Fatti che sapevo essere già avvenuti in Italia."},
    {"source": "Corbio nipote d' Ortensio menò sua vita più bassa e più viziosa",
     "reference": "Corbio, nipote di Ortensio, condusse una vita più misera e dissoluta."},
    {"source": "Altressì uno amante chiamando merzé alla sua donna dice parole e ragioni molte, et ella si difende in suo dire.",
     "reference": "Allo stesso modo, un amante rivolge suppliche alla sua donna con molte parole e argomentazioni, e lei risponde difendendosi."},
    {"source": "Io mi ricordo (ch. 347) che essendo adirato scapigliai la mia donna. Ohi, quanti dì questa ira mi tolse!",
     "reference": "Ricordo (ch. 347) che, preso dalla rabbia, le scompigliai i capelli. Ah, quanti giorni persi a causa di quell’ira!"},
    {"source": "colui del quale tu tti solevi dolere ch' era amante della tua donna;",
     "reference": "Colui di cui ti lamentavi, perché era l’amante della tua donna."},
    {"source": "Ma no sapeano già le nomora di coloro dela congiurazione, ché la donna no nominava già li nomi.",
     "reference": "Ma non conoscevano ancora i nomi dei congiurati, perché la donna non li aveva rivelati."},
    {"source": "Creti?  Certo quand'elli si mosse, elli ti dixe: \"O fedele mia donna, fa' che in mio luogo ti sia racomandato il nostro hoste troiano\".",
     "reference": "Lo credi? Certo, quando partì, ti disse: «O mia fedele donna, fa’ che il nostro ospite troiano sia raccomandato da te al posto mio»."},
    {"source": "A Milano fue ripressa la malvagità d' una donna in simile bugìa, nel tempo medesimo di questo signore della republica, in questo modo: ",
     "reference": "A Milano, nello stesso periodo di quel signore della repubblica, fu scoperta la malvagità di una donna per una bugia simile, in questo modo:"},
    {"source": "E lamentavansi dell'iniquità d'Appio, e ripiagnevano la malavventurata beltà della pulcella e la necessità del padre.",
     "reference": "Si lamentavano dell’ingiustizia di Appio e tornavano a piangere per la sventurata bellezza della fanciulla e la costrizione del padre."}
]


f1 = '/content/drive/MyDrive/Previti/gold_reference.jsonl'
f2 = '/content/drive/MyDrive/Previti/adaptive_gold_reference.jsonl'

# Write the jsonl file.
with open(f1, 'w', encoding='utf-8') as f:
    for entry in gold_reference:
        json_line = json.dumps(entry, ensure_ascii=False)
        f.write(json_line + '\n')

# Write the jsonl file.
with open(f2, 'w', encoding='utf-8') as f:
    for entry in adaptive_gold_reference:
        json_line = json.dumps(entry, ensure_ascii=False)
        f.write(json_line + '\n')



### 4.2: JUDGE FUNCTION

In the judging criteria we build up two differrent rubrics,based on two types of criteria:
- **Literal-criteria** : in this set, the focus is primarly on the word choiche,rewarding higher a literal and structural faithfullness.
- **Adaptive-criteria** : here the focus is more on the meaning,where a rephrasing is not particularly penalized if it has been crucial for the global sense.

In [ ]:
# Load model and tokenizer
model_id = "prometheus-eval/prometheus-7b-v2.0"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quant_config,
)
model.eval()

In [ ]:
Prompt = '''###Task Description:
An instruction (the original archaic sentence), a response (translation to evaluate), a reference answer that gets a score of 5, and a score rubric representing an evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: (write a feedback for criteria). [RESULT] (an integer number between 1 and 5)\"
4. Generate a **short** and **coincise** feedback avoiding redunancy and limiting the lenght strictly.
5. Please do not generate any other opening, closing, and explanations.

###The instruction to evaluate:
{orig_instruction}

###Response to evaluate:
{orig_response}

###Reference Answer (Score 5):
{orig_reference_answer}

###Score Rubrics:
{orig_criteria}
Score 1: {orig_score1_description}
Score 2: {orig_score2_description}
Score 3: {orig_score3_description}
Score 4: {orig_score4_description}
Score 5: {orig_score5_description}

###Feedback:'''


# Function to run the judge.
def judge(row,adaptation_criteria):

  if adaptation_criteria:
    orig_criteria = "Naturalness and fluency in the translation from archaic to modern Italian."
    orig_score1_description = "Completely unacceptable.The translation is totally out of context with no logic."
    orig_score2_description = "Poor translation.The generated sentence contains critical semantic errors, omissions or unjustified additions. It doesn't sound natural."
    orig_score3_description = "Partially wrong translation.The global meaning is almost the same but the translation has slightly misleading interpretations."
    orig_score4_description = "Good translation.The generated text is accurate and well adapted to modern language, with only minor issues in the naturalness of the phrasing."
    orig_score5_description = "Perfect translation.The translation is faithful to the original meaning, clear and natural. Fluent and accurate in modern Italian."
  else:
    orig_criteria = "Structural fidelity, grammatical and lexical accuracy in the translation from archaic to modern Italian."
    orig_score1_description = "Completely unacceptable.The generated text makes no sense with inappropriate word choices and unacceptable grammatical errors."
    orig_score2_description = "Poor translation.The translation has critical lexical and grammatical errors,omissions or unjustified additions."
    orig_score3_description = "Partially wrong translation.The basic idea is almost the same but the translation contains clear inaccuracies like imprecise word choices or minor grammatical errors."
    orig_score4_description = "Good translation.The translation is right and faithful to the original sentence, but the structure slightly diverges. Still grammatically and lexically correct."
    orig_score5_description = "Perfect translation.The generated sentence is faithful to the original structure, grammatically and lexically perfect with no omissions. Every word choice is perfect and coherent."

  # Building the prompt.
  prompt = Prompt.format(
        orig_instruction=row["source"],
        orig_response=row["translation"],
        orig_reference_answer=row["reference_answer"],
        orig_criteria=orig_criteria,
        orig_score1_description=orig_score1_description,
        orig_score2_description=orig_score2_description,
        orig_score3_description=orig_score3_description,
        orig_score4_description=orig_score4_description,
        orig_score5_description=orig_score5_description,

    )
  # Build the conv
  conv = get_conv_template("mistral")
  conv.set_system_message(
      "You are a fair judge assistant whose task is to evaluate translations from archaic Italian into clear, fluent modern Italian. "
      "Provide clear, objective feedback based strictly on the provided rubric, ensuring each assessment reflects the absolute standards set for performance. "
  )

  conv.append_message(conv.roles[1], prompt)
  prompt = conv.get_prompt()

  # Tokenizer and generation.
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
  judge_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return judge_output.split("###Feedback:")[-1].strip()



### 4.3: JUDGING M2M100


#### 4.3.1: M2M100-> LITERAL CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100-judge-literal_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=False), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100-judge-literal_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: E dall'altra parte Aiaces era un cavaliere franco e prodo alle armi, di grande fortuna, ma non era pieno di grande seno.
JUDGEMENT  : The translation provided is a direct translation of the original sentence, maintaining the structure and meaning. However, there is a slight deviation in the choice of words. The phrase "ben fatta l' opera" is translated as "ben condotta, l’opera riuscì, ecc." in the reference answer, which is more accurate and idiomatic in modern Italian. The phrase "ben fatta" is a more common expression in Italian, and "ben condotta" is a more formal and less commonly used expression. The phrase "di gran guisa" is also translated as "di gran fortuna" in the response, which is a less common expression in modern Italian. Despite these minor inaccuracies, the translation is still gra

#### 4.3.2: M2M100->ADAPTIVE CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/adaptive_gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100-judge-adaptive_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=True), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100-judge-adaptive_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: E dall'altra parte Aiaces era un cavaliere franco e prodo alle armi, di grande fortuna, ma non era pieno di grande seno.
JUDGEMENT  : The translation provided is a direct translation from the archaic Italian sentence, which results in a sentence that is not entirely natural in modern Italian. The phrase "ben fatta l' opera perché etc." is translated as "di grande fortuna," which is a literal translation and does not convey the intended meaning of the original sentence. The original sentence implies that the war was successful, but the translation does not capture this nuance. Additionally, the phrase "di gran guisa" is translated as "di grande fortuna," which is a significant deviation from the original meaning. The original sentence suggests that the war was successful due to the efforts of Aiaces

### 4.4: JUDGING mBART-50

#### 4.4.1: mBART-50->LITERAL CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50-judge-literal_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=False), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50-judge-literal_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: La guerra ha fatto bene, perché etc. E dall'altra parte Aiaces era un bravo guerriero, e brava per le armi, ma non pieno di grande senno.
JUDGEMENT  : The translation provided is a simplified version of the original sentence. It loses some of the nuances and specificity of the original text. For instance, the phrase "quella guerra ben fatta l' opera perché etc." is translated as "La guerra ha fatto bene, perché etc." which loses the specificity of "ben fatta" and "l' opera riuscì". The phrase "Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno" is translated as "Aiaces era un bravo guerriero, e brava per le armi, ma non pieno di grande senno." which loses the specificity of "cavaliere franco" and "prode all' arme". The translation also loses the speci

#### 4.4.2: mBART-50->ADAPTIVE CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/adaptive_gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50-judge-adaptive_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=True), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50-judge-adaptive_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: La guerra ha fatto bene, perché etc. E dall'altra parte Aiaces era un bravo guerriero, e brava per le armi, ma non pieno di grande senno.
JUDGEMENT  : The translation provided is a simplified version of the original sentence. It loses some of the nuances and specificity of the original text. For instance, "quella guerra ben fatta" is translated as "La guerra ha fatto bene," which is a more general statement. The phrase "perché etc." is also simplified to "perché etc." which doesn't convey the same level of detail as the original. Additionally, "Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno" is translated as "Aiaces era un bravo guerriero, e brava per le armi, ma non pieno di grande senno." This translation loses the specificity of Aiaces being a 

### 4.5: JUDGING MISTRAL-7B INSTRUCT V-03

#### 4.5.1: MISTRAL-7B INSTRUCT V-03 VANILLA ZERO SHOT->LITERAL CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot-judge-literal_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=False), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot-judge-literal_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella guerra fu bene condotta per ragioni complesse et Aiace era un cavaliere francese coraggioso e abile nell'utilizzo delle armi, ma mancava di profonda saggezza
JUDGEMENT  : The translation provided is largely accurate and maintains the original meaning. However, there are minor grammatical errors and slight deviations from the original structure. For instance, the phrase "fu bene condotta" is a more modern way of saying "fu ben condotta," which is more faithful to the original archaic Italian. Additionally, the phrase "un cavaliere francese coraggioso e abile nell'utilizzo delle armi" is a bit more elaborate than necessary, which slightly deviates from the original sentence's simplicity. Despite these minor issues, the translation is still grammatically and lexically correct, and the overall m

#### 4.5.2: MISTRAL-7B INSTRUCT V-03 VANILLA ZERO SHOT->ADAPTIVE CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/adaptive_gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot-judge-adaptive_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=True), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot-judge-adaptive_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella guerra fu bene condotta per ragioni complesse et Aiace era un cavaliere francese coraggioso e abile nell'utilizzo delle armi, ma mancava di profonda saggezza
JUDGEMENT  : The translation provided is generally accurate and captures the essence of the original archaic Italian sentence. However, there are minor issues in the naturalness of the phrasing. For instance, the phrase "fu bene condotta" could be more naturally expressed as "fu portata a termine con successo" to better reflect the original meaning. Additionally, the phrase "era un cavaliere francese coraggioso e abile nell'utilizzo delle armi" could be more concisely expressed as "era un cavaliere leale e coraggioso in battaglia" to maintain the original's brevity and clarity. Despite these minor issues, the overall translation is clea

#### 4.5.3: MISTRAL-7B INSTRUCT V-03 VANILLA FIVE SHOT->LITERAL CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot-judge-literal_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=False), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot-judge-literal_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella guerra fu condotta con grande abilità, poiché [l'azione] aveva una buona ragione. Da un lato c'era Achille, un cavalieri francese valente e coraggioso nelle armi, molto capace, ma non troppo astuto.
JUDGEMENT  : The translation provided is largely accurate and maintains the original meaning. However, there are minor grammatical errors and a slight deviation from the original structure. For instance, the phrase "poiché [l'azione]" is not grammatically correct in modern Italian, and the sentence "Da un lato c'era Achille, un cavalieri francese valente e coraggioso nelle armi, molto capace, ma non troppo astuto" is a bit more complex than necessary, which could potentially confuse the reader. Despite these issues, the translation still conveys the main ideas of the original sentence. Therefore,

#### 4.5.4: MISTRAL-7B INSTRUCT V-03 VANILLA FIVE SHOT->ADAPTIVE CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/adaptive_gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot-judge-adaptive_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=True), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot-judge-adaptive_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella guerra fu condotta con grande abilità, poiché [l'azione] aveva una buona ragione. Da un lato c'era Achille, un cavalieri francese valente e coraggioso nelle armi, molto capace, ma non troppo astuto.
JUDGEMENT  : The translation provided captures the essence of the original archaic Italian sentence, but there are slight inaccuracies that affect the overall fluency and naturalness of the modern Italian text. The phrase "Quella guerra fu condotta con grande abilità" is a direct translation of the original sentence, which is correct. However, the phrase "poiché [l'azione] aveva una buona ragione" is a bit ambiguous and does not convey the same meaning as the original sentence. The original sentence implies that the war was successful because of the opera, which is not clearly communicated in the

#### 4.5.5: MISTRAL-7B INSTRUCT V-03 REASONING FIVE SHOT->LITERAL CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot-judge-literal_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=False), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot-judge-literal_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella guerra fu combattuta per ragioni importanti ed efficaci. Tuttavia, dalla parte opposta vi era Achille, un cavalieri francese coraggioso e molto abilmente armato, ma non sempre intelligente.
JUDGEMENT  : The translation provided is largely accurate and maintains the original meaning. However, there are minor inaccuracies that affect the overall quality of the translation. The phrase "fu combattuta per ragioni importanti ed efficaci" is a slight deviation from the original "fu ben condotta". The use of "fu combattuta" instead of "fu ben condotta" changes the tone from "was well conducted" to "was fought", which is not as precise. Additionally, the phrase "un cavalieri francese coraggioso e molto abilmente armato" is a bit more elaborate than the original "un cavaliere franco e prode all'arme".

#### 4.5.6: MISTRAL-7B INSTRUCT V-03 REASONING FIVE SHOT->ADAPTIVE CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/adaptive_gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot-judge-adaptive_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=True), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot-judge-adaptive_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella guerra fu combattuta per ragioni importanti ed efficaci. Tuttavia, dalla parte opposta vi era Achille, un cavalieri francese coraggioso e molto abilmente armato, ma non sempre intelligente.
JUDGEMENT  : The translation provided captures the essence of the original archaic Italian sentence, but there are slight inaccuracies that affect the overall fluency and naturalness of the modern Italian text. The phrase "fu combattuta per ragioni importanti ed efficaci" is a bit too formal and lacks the simplicity and directness of the original sentence. Additionally, the use of "vi era Achille" instead of "era un cavaliere francese" introduces a slight redundancy, which is not present in the original sentence. The phrase "un cavalieri francese coraggioso e molto abilmente armato" is a bit too detailed 

### 4.6: JUDGING ZEPHYR-7B BETA

#### 4.6.1: ZEPHYR-7B BETA VANILLA THREE SHOT->LITERAL CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot-judge-literal_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=False), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot-judge-literal_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella Guerra fu un'operazione molto bene realizzata e la sua efficacia deriva da vari motivi, tra cui ... E d'altro canto Èaco, un cavaliere francese, era un uomo valoroso e pronto alla battaglia, di grandissimo valore militare, ma non raggiungeva la massima saggezza intellettuale.
JUDGEMENT  : The translation provided is largely accurate and maintains the original meaning. However, there are minor grammatical errors and a slight deviation from the original structure. For instance, the phrase "Quella Guerra fu un'operazione molto bene realizzata" is grammatically incorrect as it should be "Quella guerra fu ben condotta". Additionally, the phrase "Èaco, un cavaliere francese, era un uomo valoroso e pronto alla battaglia" is a bit more elaborate than necessary, which slightly diverges from the origi

#### 4.6.2: ZEPHYR-7B BETA VANILLA THREE SHOT->ADAPTIVE CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/adaptive_gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot-judge-adaptive_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=True), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot-judge-adaptive_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quella Guerra fu un'operazione molto bene realizzata e la sua efficacia deriva da vari motivi, tra cui ... E d'altro canto Èaco, un cavaliere francese, era un uomo valoroso e pronto alla battaglia, di grandissimo valore militare, ma non raggiungeva la massima saggezza intellettuale.
JUDGEMENT  : The translation provided is largely accurate and captures the essence of the original archaic Italian sentence. However, there are minor issues that affect the naturalness of the phrasing. For instance, the phrase "fu un'operazione molto bene realizzata" could be more concise and natural, as "fu portata a termine con successo" would be more idiomatic in modern Italian. Additionally, the phrase "un uomo valoroso e pronto alla battaglia" could be simplified to "un cavaliere valoroso e pronto alla battaglia" t

#### 4.6.3: ZEPHYR-7B BETA VANILLA FIVE SHOT->LITERAL CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot-judge-literal_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=False), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot-judge-literal_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quell'ottava guerra fu una opera eccellente per cui si può dire "etc.", poiché (da parte opposta), Ettore era un valente cavalieri francese e coraggioso sul campo di battaglia, di gran lunga; tuttavia, non disponeva di grandissima saggezza.
JUDGEMENT  : The translation provided is largely accurate and maintains the original meaning. However, there are minor issues that prevent it from achieving a perfect score. The phrase "Quell'ottava guerra fu una opera eccellente per cui si può dire 'etc.' poiché" is a bit awkward and could be more naturally phrased. Additionally, the phrase "Ettore era un valente cavalieri francese e coraggioso sul campo di battaglia, di gran lunga" is a bit redundant, as "valente" and "coraggioso" are synonyms. The translation also lacks the original sentence's structure, whic

#### 4.6.4: ZEPHYR-7B BETA VANILLA FIVE SHOT->ADAPTIVE CRITERIA




In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/adaptive_gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot-judge-adaptive_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=True), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot-judge-adaptive_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quell'ottava guerra fu una opera eccellente per cui si può dire "etc.", poiché (da parte opposta), Ettore era un valente cavalieri francese e coraggioso sul campo di battaglia, di gran lunga; tuttavia, non disponeva di grandissima saggezza.
JUDGEMENT  : The translation provided is a good attempt at modernizing the archaic Italian sentence. It captures the essence of the original text, maintaining the context and the meaning of the words. However, there are minor issues in the naturalness of the phrasing. For instance, the phrase "fu una opera eccellente per cui si può dire 'etc.'", while accurate, could be more natural in modern Italian. The use of "valente cavalieri francese" and "coraggioso sul campo di battaglia" are also slightly more formal than necessary, which could affect the overall fluenc

#### 4.6.5: ZEPHYR-7B BETA REASONING FIVE SHOT->LITERAL CRITERIA

In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot-judge-literal_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=False), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot-judge-literal_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quell'operazione bellica fu efficacemente condotta; ecco dalla parte opposta vi era Ayace, cavaliero francese valorosissimo nelle armi, molto deciso nel servizio, ma non dotato di grande saggezza mentale.
JUDGEMENT  : The translation provided is largely accurate and maintains the original meaning. However, there are minor grammatical errors and slight deviations from the original structure. For instance, the phrase "Quell'operazione bellica fu efficacemente condotta" is grammatically correct but the word "efficacemente" is not commonly used in modern Italian, which could cause confusion. Additionally, the phrase "vi era Ayace, cavaliero francese valorosissimo nelle armi" is a bit more formal than the original sentence, which could be seen as a slight deviation from the original tone. Despite these 

#### 4.6.6: ZEPHYR-7B BETA REASONING FIVE SHOT->ADAPTIVE CRITERIA




In [ ]:

# Extract the first 20 rows.
input_path  = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot.jsonl"
df = pd.read_json(input_path, lines=True)
df_transl = df.head(20)

# Extract gold reference
reference_path = "/content/drive/MyDrive/Previti/adaptive_gold_reference.jsonl"
reference_df = pd.read_json(reference_path, lines=True)
rederence_df = reference_df.head(20)

# Merge dataset.
df = (
    df_transl
      .merge(reference_df[['source', 'reference']], on='source', how='left')
      .rename(columns={'reference': 'reference_answer'})
)


# Save results
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot-judge-adaptive_criteria.jsonl"
df["judgement"] = df.apply(lambda row: judge(row, adaptation_criteria=True), axis=1)
df.drop(columns=["reference_answer"], inplace=True)
df.to_json(output_path, orient="records", lines=True, force_ascii=False)


In [ ]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot-judge-adaptive_criteria.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print(f"JUDGEMENT  : {row['judgement']}")

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: Quell'operazione bellica fu efficacemente condotta; ecco dalla parte opposta vi era Ayace, cavaliero francese valorosissimo nelle armi, molto deciso nel servizio, ma non dotato di grande saggezza mentale.
JUDGEMENT  : The translation provided is largely accurate and captures the essence of the original archaic Italian sentence. The use of modern Italian terms such as "operazione bellica" and "valorosissimo" is appropriate and contributes to the overall fluency of the text. However, the phrase "molto deciso nel servizio" could be more naturally expressed as "molto deciso in servizio" to better reflect the original meaning. Additionally, the phrase "dotato di grande saggezza mentale" could be simplified to "non particolarmente saggio" to maintain the original tone and meaning. These minor issues in p

# 5: HUMAN METRICS CORRELATION

## 5.1: HUMAN ANNOTATIONS

In this section will be stored the human annotations written by us of the first 20 translations.

For each model there will be the version for the literal criteria and the version for the adaptive criteria

In [ ]:
#M2M100
M2M100_literal_criteria = [3, 3, 5, 4, 3, 4, 2, 4, 3, 2, 3, 5, 4, 3, 4, 4, 5, 3, 4, 3]
M2M100_adaptive_criteria = 	[3, 2, 4, 3, 2, 4, 2, 3, 3, 3, 2, 3, 5, 3, 3, 3, 4, 4, 3, 2]

#mBART-50
mBART_50_literal_criteria = 	[4, 3, 3, 4, 3, 3, 1, 2, 3, 4, 3, 5, 3, 4, 2, 1, 3, 4, 5, 4]
mBART_50_adaptive_criteria = 	[3, 3, 4, 4, 3, 3, 1, 2, 3, 3, 2, 3, 2, 3, 2, 2, 1, 4, 3, 4]

# MISTRAL 7B INSTRUCT V03
mistral_vanilla_zero_shot_literal_criteria =  	[4, 3, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4]
mistral_vanilla_zero_shot_adaptive_criteria  = 	[4, 3, 5, 3, 4, 3, 3, 4, 4, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4]

mistral_vanilla_five_shot_literal_criteria = [4, 3, 4, 4, 4, 1, 4, 3, 3, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4]
mistral_vanilla_five_shot_adaptive_criteria = 	[4, 3, 3, 4, 4, 2, 4, 3, 3, 3, 4, 3, 4, 4, 4, 4, 5, 4, 4, 4]

mistral_reasoning_five_shot_literal_criteria = [4, 3, 3, 4, 4, 3, 3, 2, 3, 2, 4, 3, 3, 3, 3, 4, 4, 1, 4, 4]
mistral_reasoning_five_shot_adaptive_criteria = [4, 4, 3, 4, 4, 4, 3, 3, 5, 3, 4, 4, 3, 4, 3, 3, 4, 2, 4, 4]

# ZEPHYR 7B BETA
zephyr_vanilla_three_shot_literal_criteria =  	[3, 3, 3, 4, 3, 3, 3, 1, 4, 4, 3, 4, 1, 4, 4, 4, 4, 4, 1, 4]
zephyr_vanilla_three_shot_adaptive_criteria = 	[4, 4, 3, 4, 4, 3, 4, 1, 3, 3, 3, 4, 2, 3, 4, 3, 4, 4, 1, 4]

zephyr_vanilla_five_shot_literal_criteria = 	[3, 3, 4, 4, 4, 3, 3, 3, 3, 2, 4, 2, 3, 4, 3, 5, 4, 4, 4, 4]
zephyr_vanilla_five_shot_adaptive_criteria = [4, 4, 4, 4, 4, 3, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 3, 4, 4]

zephyr_reasoning_five_shot_literal_criteria = 	[4, 4, 3, 4, 4, 4, 3, 4, 2, 3, 2, 3, 4, 4, 4, 5, 4, 4, 4, 3]
zephyr_reasoning_five_shot_adaptive_criteria = 	[4, 4, 3, 4, 4, 3, 4, 3, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 3]

## 5.2: PROMETHEUS RESULT EXTRACTION

In [ ]:
# Utility to exctact prometheus result.
def get_vector(jsonl_path):
    df = pd.read_json(jsonl_path, lines=True)
    df['result'] = df['judgement'].apply(lambda text: int(re.search(r"\[RESULT\]\s*(\d+)", text).group(1)) if re.search(r"\[RESULT\]\s*(\d+)", text) else None)
    return df['result'].head(20).tolist()

In [ ]:
# M2M100.
prom_M2M100_literal_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100-judge-literal_criteria.jsonl")
prom_M2M100_adaptive_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100-judge-adaptive_criteria.jsonl")

# mBART-50.
prom_mBART_50_literal_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50-judge-literal_criteria.jsonl")
prom_mBART_50_adaptive_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50-judge-adaptive_criteria.jsonl")

# MISTRAL 7B INSTRUCT V03.
prom_mistral_vanilla_zero_shot_literal_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot-judge-literal_criteria.jsonl")
prom_mistral_vanilla_zero_shot_adaptive_criteria  = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_zero_shot-judge-adaptive_criteria.jsonl")

prom_mistral_vanilla_five_shot_literal_criteria =  get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot-judge-literal_criteria.jsonl")
prom_mistral_vanilla_five_shot_adaptive_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-vanilla_five_shot-judge-adaptive_criteria.jsonl")

prom_mistral_reasoning_five_shot_literal_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot-judge-literal_criteria.jsonl")
prom_mistral_reasoning_five_shot_adaptive_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-mistral7b_instruct_v03-reasoning_five_shot-judge-adaptive_criteria.jsonl")

# ZEPHYR 7B BETA.
prom_zephyr_vanilla_three_shot_literal_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot-judge-literal_criteria.jsonl")
prom_zephyr_vanilla_three_shot_adaptive_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_three_shot-judge-adaptive_criteria.jsonl")

prom_zephyr_vanilla_five_shot_literal_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot-judge-literal_criteria.jsonl")
prom_zephyr_vanilla_five_shot_adaptive_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-vanilla_five_shot-judge-adaptive_criteria.jsonl")

prom_zephyr_reasoning_five_shot_literal_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot-judge-literal_criteria.jsonl")
prom_zephyr_reasoning_five_shot_adaptive_criteria = get_vector("/content/drive/MyDrive/Previti/Previti-hw2_transl-zephyr_7b_beta-reasoning_five_shot-judge-adaptive_criteria.jsonl")



## 5.3: HUMAN METRICS CORRELATION

In this final section it is performed the agreement between the human annotations and the prometheus result computing di accuracy score and the Cohen’s Kappa Coefficient.

In [ ]:
# Utility for the evaluation.
def evaluate_agreement(human_scores, prom_scores):

    if len(human_scores) != len(prom_scores):
        raise ValueError("Different size.")

    accuracy = accuracy_score(human_scores, prom_scores)
    kappa = cohen_kappa_score(human_scores, prom_scores, weights="quadratic")

    return {"Accuracy": accuracy, "Cohen_kappa": kappa}


In [ ]:
# M2M100.
M2M100_literal_criteria_agreement = evaluate_agreement(M2M100_literal_criteria, prom_M2M100_literal_criteria)
M2M100_adaptive_criteria_agreement = evaluate_agreement(M2M100_adaptive_criteria, prom_M2M100_adaptive_criteria)

# mBART-50.
mBART_50_literal_criteria_agreement = evaluate_agreement(mBART_50_literal_criteria, prom_mBART_50_literal_criteria)
mBART_50_adaptive_criteria_agreement = evaluate_agreement(mBART_50_adaptive_criteria, prom_mBART_50_adaptive_criteria)

# MISTRAL 7B INSTRUCT V03
mistral_vanilla_zero_shot_literal_criteria_agreement = evaluate_agreement(mistral_vanilla_zero_shot_literal_criteria, prom_mistral_vanilla_zero_shot_literal_criteria)
mistral_vanilla_zero_shot_adaptive_criteria_agreement = evaluate_agreement(mistral_vanilla_zero_shot_adaptive_criteria, prom_mistral_vanilla_zero_shot_adaptive_criteria)

mistral_vanilla_five_shot_literal_criteria_agreement = evaluate_agreement(mistral_vanilla_five_shot_literal_criteria, prom_mistral_vanilla_five_shot_literal_criteria)
mistral_vanilla_five_shot_adaptive_criteria_agreement = evaluate_agreement(mistral_vanilla_five_shot_adaptive_criteria, prom_mistral_vanilla_five_shot_adaptive_criteria)

mistral_reasoning_five_shot_literal_criteria_agreement = evaluate_agreement(mistral_reasoning_five_shot_literal_criteria, prom_mistral_reasoning_five_shot_literal_criteria)
mistral_reasoning_five_shot_adaptive_criteria_agreement = evaluate_agreement(mistral_reasoning_five_shot_adaptive_criteria, prom_mistral_reasoning_five_shot_adaptive_criteria)

# ZEPHYR 7B
zephyr_vanilla_three_shot_literal_criteria_agreement = evaluate_agreement(zephyr_vanilla_three_shot_literal_criteria, prom_zephyr_vanilla_three_shot_literal_criteria)
zephyr_vanilla_three_shot_adaptive_criteria_agreement = evaluate_agreement(zephyr_vanilla_three_shot_adaptive_criteria, prom_zephyr_vanilla_three_shot_adaptive_criteria)

zephyr_vanilla_five_shot_literal_criteria_agreement = evaluate_agreement(zephyr_vanilla_five_shot_literal_criteria, prom_zephyr_vanilla_five_shot_literal_criteria)
zephyr_vanilla_five_shot_adaptive_criteria_agreement = evaluate_agreement(zephyr_vanilla_five_shot_adaptive_criteria, prom_zephyr_vanilla_five_shot_adaptive_criteria)

zephyr_reasoning_five_shot_literal_criteria_agreement = evaluate_agreement(zephyr_reasoning_five_shot_literal_criteria, prom_zephyr_reasoning_five_shot_literal_criteria)
zephyr_reasoning_five_shot_adaptive_criteria_agreement = evaluate_agreement(zephyr_reasoning_five_shot_adaptive_criteria, prom_zephyr_reasoning_five_shot_adaptive_criteria)


In [ ]:
results = {
    "M2M100_literal": M2M100_literal_criteria_agreement,
    "M2M100_adaptive": M2M100_adaptive_criteria_agreement,

    "mBART50_literal": mBART_50_literal_criteria_agreement,
    "mBART50_adaptive": mBART_50_adaptive_criteria_agreement,

    "Mistral_zero_literal": mistral_vanilla_zero_shot_literal_criteria_agreement,
    "Mistral_zero_adaptive": mistral_vanilla_zero_shot_adaptive_criteria_agreement,

    "Mistral_five_literal": mistral_vanilla_five_shot_literal_criteria_agreement,
    "Mistral_five_adaptive": mistral_vanilla_five_shot_adaptive_criteria_agreement,

    "Mistral_reasoning_literal": mistral_reasoning_five_shot_literal_criteria_agreement,
    "Mistral_reasoning_adaptive": mistral_reasoning_five_shot_adaptive_criteria_agreement,

    "Zephyr_three_literal": zephyr_vanilla_three_shot_literal_criteria_agreement,
    "Zephyr_three_adaptive": zephyr_vanilla_three_shot_adaptive_criteria_agreement,

    "Zephyr_five_literal": zephyr_vanilla_five_shot_literal_criteria_agreement,
    "Zephyr_five_adaptive": zephyr_vanilla_five_shot_adaptive_criteria_agreement,

    "Zephyr_reasoning_literal": zephyr_reasoning_five_shot_literal_criteria_agreement,
    "Zephyr_reasoning_adaptive": zephyr_reasoning_five_shot_adaptive_criteria_agreement,
}

# Final print
df_results = pd.DataFrame(results).T
df_results = df_results.sort_index()
print(df_results.round(3))

                            Accuracy  Cohen_kappa
M2M100_adaptive                 0.45        0.649
M2M100_literal                  0.55        0.737
Mistral_five_adaptive           0.60        0.626
Mistral_five_literal            0.65        0.701
Mistral_reasoning_adaptive      0.75        0.667
Mistral_reasoning_literal       0.65        0.815
Mistral_zero_adaptive           0.75        0.797
Mistral_zero_literal            0.90        0.808
Zephyr_five_adaptive            0.55        0.416
Zephyr_five_literal             0.60        0.698
Zephyr_reasoning_adaptive       0.65        0.512
Zephyr_reasoning_literal        0.80        0.632
Zephyr_three_adaptive           0.60        0.457
Zephyr_three_literal            0.65        0.574
mBART50_adaptive                0.60        0.694
mBART50_literal                 0.55        0.609
